First, we manage all imports for the code.

In [3]:
!pip install gensim

Defaulting to user installation because normal site-packages is not writeable


In [4]:
#imports
import pandas as pd
import numpy as np
import random
import ast
from torch import nn
import torch
import re
import nltk
from torch.utils.data import random_split, Subset, DataLoader
import gensim
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

import zipfile
import os
import shutil

/home/alfrodo/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Next we process the zipped data to csv. It is stored in .zip, because it is smaller on git and then locally it is extracted to csv.

In [5]:
with zipfile.ZipFile("data/dataset.zip", "r") as zip_ref:
    zip_ref.extractall("data")

for file_name in os.listdir("data/dataset"):
    source = "data/dataset/" + file_name
    destination = "data/" + file_name
    if os.path.isfile(source): shutil.move(source, destination)

os.rmdir("data/dataset")
del source, destination

Preprocessing of the datasets, the goal is to receive a table with the following columns: Name, Lyrics, Genre<br>
df Dataset - Name, Lyrics, Genre<br>
df3 Dataset - 10000 entrys of random selectet lyrics to train doc2vec

In [7]:

df = pd.read_csv('data/Spotify-2000.csv')
df = df[['Title', 'Top Genre']] #take only the name and genre
df2 = pd.read_csv('data/spotify_millsongdata.csv')
df['lyrics'] = ' ' # add column lyrics
#Now combine those two documents by the title
found = 0

for x, title in enumerate(df['Title']):
    #print(title)
    for y, title2 in enumerate(df2['song']):
        if title2.lower() == title.lower():
            df['lyrics'].iloc[x] = df2['text'].iloc[y]

print("finished combining..")

finished combining..


Collect 10000 random entrys of lyrics from the millsongdata Dataset

In [9]:
df3 = df2['text'].sample(n=10000)
print(df3)
print('done')

42664    Once again we sit in silence  \r\nAfter all is...
47651    Ah, she tastes like wine  \r\nSuch a human bei...
22391    Every kiss is a goodbye kiss  \r\nAnd every to...
34894    You could have gotten to know  \r\nIn the tint...
55269    Are the Wings of Angels all a lie Tell me is t...
                               ...                        
1907     Someone found me a letter you wrote me on the ...
4126     Well if you ever plan to motor west  \r\nTrave...
33605    It's easier to tell a lie than it is to tell t...
20185    No authority!  \r\nI've spoke my mind a thousa...
31883    I want to be loved, I want to be loved  \r\nI ...
Name: text, Length: 10000, dtype: object
done


In [ ]:
#remove songs which were not in both datsets
songs_to_remove = []
for x, lyrics in enumerate(df['lyrics']):
    if lyrics == '':
        songs_to_remove.append(x)
df.drop(songs_to_remove, axis = 0, inplace = True)

print(df)

Further preprocessing of the lyrics itself. We remove the stopwords and punctuations with regex and stopwords from ntlk

In [ ]:
from nltk.corpus import stopwords
nltk.download('stopwords')

sw = stopwords.words('english')
punc_regex = r'[^\s\w]' #searches for everything thats not a word or space
stopword_regex = r'\b{0}\b'

print(df['lyrics'].iloc[1])

for x, lyrics in enumerate(df['lyrics']):
    txt = lyrics
    txt = re.sub(punc_regex, '', txt) #remove punuctuations
    for sword in sw:
        txt = re.sub(stopword_regex.format(sword), '', txt, flags=re.IGNORECASE) #remove every stopword
    df['lyrics'].iloc[x] = txt

#print(df)
print(df['lyrics'].iloc[1])

Tokenize the lyrics and create tagged Documents

In [ ]:
tagged_data = []

for i,d in enumerate(df['lyrics']):
    tokenized_words = nltk.tokenize.word_tokenize(d)
    tagged_data.append(TaggedDocument(words=tokenized_words, tags=str(i)))

print(tagged_data)

Setting up the Doc2Vec model

In [ ]:
model = gensim.models.doc2vec.Doc2Vec(vector_size=100, min_count=1, epochs=30)
model.build_vocab(tagged_data)
model.train(tagged_data, total_examples=model.corpus_count, epochs=model.epochs)

print("Doc2Vec model finished training")

Building the DataLoader for the machine learning model

In [ ]:
columns = ['Token'] + ['Target']
df_for_dataloader = pd.DataFrame(columns = columns)
df_for_dataloader.set_index(columns)

for i in range(len(tagged_data)):
    token = [model.infer_vector(tagged_data[i][0])]
    #token = [list(model.dv[i])]
    target = [5] # should be genre
    row = pd.DataFrame([token + target], columns=['Token', 'Target'])
    df_for_dataloader = pd.concat([df_for_dataloader, row])
    
display(df_for_dataloader.iloc[:5]) 

In [ ]:
device = torch.device('cpu') #torch.device('cuda' if torch.cuda.is_available() else 'cpu') For faster training in the end

class SongModel(nn.Module):
    def __init__(self, input_size, output_size):
        super(SongModel, self).__init__()
        self.input_size = input_size
        self.hidden_size = input_size * 3
        self.output_size = output_size
        self.layer_1 =   nn.Linear(self.input_size, self.hidden_size)
        self.layer_2 =   nn.Linear(self.hidden_size, self.hidden_size)
        self.layer_out = nn.Linear(self.hidden_size, self.output_size)
        self.softmax =   nn.Softmax()
        self.sigmoid =   nn.Sigmoid()
        self.tanh =      nn.Tanh()

        nn.init.xavier_uniform_(self.layer_1.weight)
        nn.init.zeros_(self.layer_1.bias)
        nn.init.xavier_uniform_(self.layer_2.weight)
        nn.init.zeros_(self.layer_2.bias)
        nn.init.xavier_uniform_(self.layer_out.weight)
        nn.init.zeros_(self.layer_out.bias)


    def forward(self, d):
        x = self.sigmoid(self.layer_1(d))
        x = self.tanh(self.layer_2(x))
        x = self.softmax(self.layer_out(x))
        return x

model = SongModel(1, 3).to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=0.025)
loss_fn = torch.nn.CrossEntropyLoss()

In [ ]:
def train(dataloader, model, loss_fn, optimizer):
    total_loss = 0
    iterations = 0
    for sample in dataloader:
        model_input = sample[0]
        should = sample[1]
        predict = model(model_input)
        loss = loss_fn(predict, should.float())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss
        iterations += 1
        if iterations % 1000 == 0:
            print(f"Predict: {predict}, Loss: {loss}")

    print(total_loss / len(dataloader))
    return